In [1]:
# Import all relevant libraries
import warnings
def fxn(): 
	warnings.warn("deprecated",DeprecationWarning)

with warnings.catch_warnings( ):
    warnings.simplefilter("ignore")
    fxn( )

# Keras imports
import keras
from keras.models import Sequential, Model
from keras.layers import Input, UpSampling1D, BatchNormalization, MaxPooling1D, MaxPooling2D, Permute, Flatten, Softmax, Dense, Dropout, Conv1D, Conv2D, Conv2DTranspose, AveragePooling1D, AveragePooling2D, Activation, Reshape
from keras.utils import np_utils
from keras.optimizers import Adam

# Other
import numpy as np
import h5py
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Load data from specific trial
def get_trial(trial_num):    
    trial = h5py.File('../data/A0' + str(trial_num) + 'T_slice.mat', 'r')
    X = np.copy(trial['image'])
    y = np.copy(trial['type'])
    y = y[0,0:X.shape[0]:1]
    y = np.asarray(y, dtype=np.int32)
    y -= 769                            # shift class labels to [0-3]
    X = np.nan_to_num(X)[:, :22, :]     # remove EOG channels
    return X, y

def get_all_trials():
    X_total = np.concatenate([get_trial(trial_num)[0] for trial_num in range(1, 10)], axis=0)
    y_total = np.concatenate([get_trial(trial_num)[1] for trial_num in range(1, 10)], axis=0)
    return X_total, y_total

def stratified_CV_split(X, y, k):
    ''' Returns a stratified train/test split, for k number of splits. Split amount
    is based on k (equal splits).
    Return value is in the form [(train indices, test indices), ... for k folds ]
    '''
    skf = StratifiedKFold(n_splits=k)
    return skf.split(X, y)

def stratified_train_test_split(X, y, k, num_trials):
    ''' Returns a stratified train/test split, for k number of splits. Test size is 
    50 * number of trials X is composed of.
    Return value is in the form [(train indices, test indices), ... for k folds ]
    '''
    sss = StratifiedShuffleSplit(n_splits=k, test_size=50*num_trials)
    return sss.split(X, y)

In [3]:
# Get the data from all the people and generate train/test split
X, y = get_all_trials()
y_cat = keras.utils.to_categorical(y, num_classes=4)
#tt_splits = stratified_train_test_split(X, y, 5)
X_train, X_test, y_train, y_test = train_test_split(X, y )

one_hot_train = keras.utils.to_categorical(y_train)
one_hot_test = keras.utils.to_categorical(y_test)


# The data for each trial is of the shape (288, 22, 1000)
#   There are 288 samples per trial (12 of each class per "run", 4 classes, 6 "runs" 
#                                   at different time periods of the day)
#   There are 22 electrodes from the EEG (represents spatial aspect of the signals)
#   There are 1000 time units (4 seconds of data, sampled at 250Hz). The first 250 units
#                                   are when no movement occurs (but the cue is heard) and
#                                   the next 750 units are when the movement occurs
# The labels for each trial belong in one of 4 classes
#   0 - left
#   1 - right
#   2 - foot
#   3 - tongue

In [33]:
x = Input(shape=(1000,22))
conv_1 = (Conv1D(filters = 10, kernel_size = 1, activation = 'elu', input_shape = (1000,22)))(x)


flattened = Flatten()(conv_1)
h = Dense(500, activation='relu')(flattened)
mu = Dense(100)(h)
log_sigma = Dense(100)(h)



def sample_z(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=(m, n_z), mean=0., std=1.)
    return mu + K.exp(log_sigma / 2) * eps

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling)([mu, log_sigma])
print(z.shape)
# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
print(h_decoded.shape)
x_decoded_mean = decoder_mean(h_decoded)
print(x_decoded_mean.shape)

# instantiate VAE model
vae = Model(x, x_decoded_mean)


# Compute VAE loss

def vae_loss(y_true, y_pred):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z,y)]
    recon = K.sum(K.binary_crossentropy(y_pred, y_true), axis=1)
    # D_KL(Q(z|X,y) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * K.sum(K.exp(log_sigma) + K.square(mu) - 1. - log_sigma, axis=1)

    return recon + kl



vae.compile(optimizer='adam', loss = vae_loss)
vae.summary()




x_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
print(len(X_train))
x_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

vae.fit(x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()

# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)





"""
# display a 2D manifold of the digits MNIST
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = generator.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()
"""

(?, 1000, 10)
(?, 500)
(?, 100)
(?, 100)


ValueError: Dimensions must be equal, but are 100 and 2 for 'lambda_27/mul' (op: 'Mul') with input shapes: [?,100], [?,2].